In [ ]:
 !pip install -q tensorflow-model-optimization


In [ ]:
import tempfile
import os
import pickle as pk
import tensorflow as tf
import numpy as np

from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History 
history = History()

np.random.seed(25)

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(5, 5), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=12,kernel_size=(3,3),activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])
model.summary()




In [ ]:
# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

unpruned_model=model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.1,
  callbacks=[history]
)

In [ ]:

dir_name = '/content/drive/My Drive/Fall2020/10617_IDL/Project/Baseline'
fname = dir_name+'Baseline.h5'
model.save(fname)

fname = 'log_dir-{d}.pkl'.format(d=folder_name)
with open(dir_name+fname,'wb') as f:
  pk.dump(logdir,f)
f.close()

metric = (unpruned_model.history['accuracy'],unpruned_model.history['val_accuracy'],
          unpruned_model.history['loss'],unpruned_model.history['val_loss'])
fname = 'metric-{d}.pkl'.format(d=folder_name)
with open(dir_name+fname,'wb') as f:
  pk.dump(metric,f)
f.close()

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 10
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                               final_sparsity=0.95,
                                                               begin_step=0,
                                                               end_step=end_step)
}

pruned_model = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

pruned_model.summary()

In [ ]:
logdir = tempfile.mkdtemp()
from keras.callbacks import History 
history = History()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  history
]
  
pruned_history=pruned_model.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)


In [ ]:
fin_sp=0.95
folder_name = str(int(fin_sp*100))
dir_name = '/content/drive/My Drive/Fall2020/10617_IDL/Project/sparse-{d}/'.format(d=folder_name)
fname = dir_name+'pruned-model-{d}.h5'.format(d=folder_name)
model_for_export.save(fname)

fname = 'log_dir-{d}.pkl'.format(d=folder_name)
with open(dir_name+fname,'wb') as f:
  pk.dump(logdir,f)
f.close()

metric = (pruned_history.history['accuracy'],pruned_history.history['val_accuracy'],
          pruned_history.history['loss'],pruned_history.history['val_loss'])
fname = 'metric-{d}.pkl'.format(d=folder_name)
with open(dir_name+fname,'wb') as f:
  pk.dump(metric,f)
f.close()

In [ ]:
def NNZ(weights):
  zeroCount = 0
  for ele in weights:
    if ele == 0.0:
      zeroCount += 1

  return zeroCount



allWeights = np.array([1],dtype=float)
for weight in model_for_export.get_weights():
  allWeights = np.concatenate((allWeights,weight.flatten()))
nnz = NNZ(allWeights[1:])
print("Number of Non-zero Params: ", nnz)
print("Sparsity - {d}%".format(d=np.round(nnz/allWeights[1:].shape[0]*100,2)))

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)



In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

In [ ]:
import matplotlib.pyplot as plt

figure,((ax1,ax2))=plt.subplots(nrows=1,ncols=2,sharex=False,sharey=False)

figure.set_figwidth(15)
sparsity=[50,80,90,95]
record={}
x=range(epochs)
for i in range(len(sparsity)):

    with open('/content/drive/My Drive/Fall2020/10617_IDL/Project/sparse-{d}/metric-{d}.pkl'.format(d=sparsity[i]), 'rb') as f:
      unpickler = pk.Unpickler(f)
      metric=unpickler.load()
      train_acc=metric[0]
      test_acc=metric[1]
    ax1.plot(x,train_acc,'*-',label='Sparsity:{d}%'.format(d=sparsity[i]))
    ax2.plot(x,test_acc,'*-',label='Sparsity:{d}%'.format(d=sparsity[i]))

ax1.set(xlabel='Epochs', ylabel='Train Accuracy')
ax2.set(xlabel='Epochs', ylabel='Test Accuracy')
ax1.legend(loc='best', prop={'size': 7})
ax2.legend(loc='best', prop={'size': 7})
plt.show()
